In [ ]:
# COVID-19 mask detector - https://github.com/alexcamargoweb/mask-detector
# Detecção de máscaras em pessoas (com ou sem)
# Adrian Rosebrock, COVID-19: Face Mask Detector with OpenCV, Keras/TensorFlow, and Deep Learning. PyImageSearch.    
# Disponível em: https://www.pyimagesearch.com/2018/09/24/opencv-face-recognition/.   
# Acessado em: 05/12/2020.  
# Arquivo: train_mask.ipynb
# Execução via Google Colab

In [ ]:
# monta o google drive
from google.colab import drive, output, files
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mask-detection
# limpa as saídas
output.clear()

In [ ]:
# importa os pacotes necessários
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [ ]:
# parâmetros de treinamento

# dataset de imagens
DATASET = '/content/drive/MyDrive/datasets'
# gráfico de treinamento
PLOT = '/content/drive/MyDrive/plots/mask_covid-19.png'
# diretório com os arquivos do detector de faces
DETECTOR = '/content/drive/MyDrive/detector'
# modelo final
MODEL = '/content/drive/MyDrive/models/mask_covid-19.model'
# confiança de detecção
CONFIDENCE = 0.5
# taxa de aprendizado
INIT_LR = 1e-4
# número de épocas
EPOCHS = 20
# batch size
BS = 32

In [ ]:
# carregamento e pré-processamento das imagens

# gera uma lista das imagens do dataset
print("[INFO] carregando imagens...")
imagePaths = list(paths.list_images(DATASET))
data = []
labels = []
# faz um loop sobre todas as imagens
for imagePath in imagePaths:
	# extrai o rótulo (classe) pelo diretório
	label = imagePath.split(os.path.sep)[-2]
	# carrega a imagem e redimensiona para um tamanho fixo (224x224)
	image = load_img(imagePath, target_size = (224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)
	# atualiza as listas de imagens e rótulos, respectivamente
	data.append(image)
	labels.append(label)
# converte as imagens e rótulos para arrays NumPy
data = np.array(data, dtype = "float32")
labels = np.array(labels)

In [ ]:
# realiza a codificação one-hot (binarização) nos rótulos 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# particiona as imagens numa divisão de 80% para treino e 20% para teste
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size = 0.20, 
																									stratify = labels, 
																									random_state = 42)
# inicializa o objeto de aumento de dados de treinamento (data augmentation) 
aug = ImageDataGenerator(rotation_range = 20, 
                         zoom_range = 0.15,	
												 width_shift_range = 0.2,
												 height_shift_range = 0.2,
												 shear_range = 0.15,
												 horizontal_flip = True,
												 fill_mode = "nearest")

In [ ]:
# transfer learning

# carrega a rede MobileNetV2
baseModel = MobileNetV2(weights = "imagenet", 
                        include_top = False, # ignora as camadas densas (FC)
												input_tensor = Input(shape=(224, 224, 3)))
baseModel.summary()

# constrói o head FC (camadas Densas) do modelo que será colocado em cima do modelo base (MobileNetV2)
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7, 7))(headModel)
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(128, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = "softmax")(headModel)

# junta as redes
model = Model(inputs = baseModel.input, outputs = headModel)

# faz um loop sobre todas as camadas no modelo base e as congela 
# para que não sejam atualizadas durante o primeiro processo de treinamento
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
# compila o modelo
print("[INFO] compilando modelo...")
opt = Adam(lr = INIT_LR, decay = INIT_LR / EPOCHS)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics = ["accuracy"])

# treina a rede (FC)
print("[INFO] treinando RNA densa...")
H = model.fit(aug.flow(trainX, trainY, batch_size = BS),
              steps_per_epoch = len(trainX) // BS,
							validation_data = (testX, testY),
							validation_steps = len(testX) // BS,
							epochs = EPOCHS)

In [ ]:
# faz as predições no conjunto de teste
print("[INFO] avaliando modelo...")
predIdxs = model.predict(testX, batch_size = BS)
# para cada imagem no conjunto de teste encontra o índice do
# rótulo com a maior probabilidade prevista correspondente
predIdxs = np.argmax(predIdxs, axis = 1)
# exibe num formato amigável
print(classification_report(testY.argmax(axis = 1), predIdxs,	
                            target_names = lb.classes_))
# salva o modelo em disco
print("[INFO] salvando modelo...")
model.save(MODEL, save_format="h5")

In [ ]:
# plota a função de perda e a acurácia de treinamento
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label = "train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label = "val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label = "train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label = "val_acc")
plt.title("Função de perda e acurácia")
plt.xlabel("Época #")
plt.ylabel("Perda/Acurácia")
plt.legend(loc = "lower left")
plt.savefig(PLOT)